#                     Predicción de Abandono de Clientes en Beta Bank



Beta Bank enfrenta una pérdida continua de clientes, lo que es más costoso que retener a los actuales. El objetivo de este proyecto es desarrollar un modelo predictivo que identifique clientes en riesgo de abandonar, utilizando datos históricos sobre su comportamiento. 

El modelo se evaluará con la métrica F1, buscando alcanzar un mínimo de 0.59, y también se medirá la AUC-ROC para asegurar la efectividad. Con estas predicciones, el banco podrá implementar estrategias de retención específicas, mejorando la lealtad y reduciendo costos.

<span style='color:#FF69B4 '> Preparación y Descarga de Datos: Procedimiento Explicado </span>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from sklearn.utils import resample

In [2]:
# Cargar el archivo CSV en un DataFrame
file_path = '/datasets/Churn.csv'
churn_data = pd.read_csv(file_path)

In [3]:
# Mostrar las primeras filas del DataFrame para una inspección inicial
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<span style='color:#1E90FF'> Explicación: </span>
    
Primero, cargaremos el archivo en un DataFrame de pandas para poder trabajar con los datos. Esto nos permitirá ver su contenido y empezar a prepararlo para el análisis.
Inspección inicial de los datos: Revisaremos la estructura del DataFrame, como las primeras filas, el tipo de datos en cada columna, y verificaremos si hay valores faltantes.
Limpieza de los datos: Si encontramos valores nulos o datos que necesitan ser formateados o corregidos, los abordaremos para asegurar que el conjunto de datos esté listo para el análisis.

In [4]:
# Paso 1: Eliminar columnas innecesarias
churn_data_cleaned = churn_data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

# Paso 2: Verificar y manejar valores nulos
churn_data_cleaned['Tenure'] = churn_data_cleaned['Tenure'].fillna(churn_data_cleaned['Tenure'].median())

# Paso 3: Convertir tipos de datos si es necesario
churn_data_cleaned = churn_data_cleaned.convert_dtypes()

# Paso 4: Codificar variables categóricas
# Convertir 'Gender' a valores numéricos (Female: 0, Male: 1)
churn_data_cleaned['Gender'] = churn_data_cleaned['Gender'].map({'Female': 0, 'Male': 1})

# Convertir 'Geography' usando variables dummy (one-hot encoding)
churn_data_cleaned = pd.get_dummies(churn_data_cleaned, columns=['Geography'], drop_first=True)

# Mostrar las primeras filas del DataFrame limpio
churn_data_cleaned.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.8,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.0,2,0,0,93826.63,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.1,0,0,1


<span style='color:#1E90FF '> Explicación: </span>

Eliminamos las columnas RowNumber, CustomerId y Surname porque no son relevantes para el análisis. Estas columnas no aportan información útil para predecir si un cliente va a abandonar el servicio.
Llenamos valores nulos en la columna Tenure con la mediana de los datos. Esto es para evitar problemas en el análisis posterior debido a valores faltantes.
Convertimos los tipos de datos si es necesario. Esto asegura que las operaciones posteriores funcionen sin errores.
Codificamos las variables categóricas (Gender y Geography). Gender se convierte a valores numéricos, y Geography se transforma en variables dummy (variables binarias), eliminando una de las categorías para evitar la multicolinealidad.

<span style='color:#FF69B4 '> Análisis del Equilibrio de Clases y Entrenamiento Inicial del Modelo </span>

In [5]:
# Revisar el equilibrio de clases en la variable objetivo
class_counts = churn_data_cleaned['Exited'].value_counts(normalize=True)
print("Distribución de clases:")
print(class_counts)

Distribución de clases:
0    0.7963
1    0.2037
Name: Exited, dtype: Float64


<span style='color:#1E90FF '> Explicación: </span>

La función value_counts(normalize=True) nos proporciona la proporción de clientes que han abandonado (valor 1) y los que no (valor 0). Esto nos permitirá ver si hay un desequilibrio significativo en las clases.

In [6]:
# Separar las características (X) y la variable objetivo (y)
X = churn_data_cleaned.drop(columns=['Exited'])
y = churn_data_cleaned['Exited'].astype(int)  # Convertir la variable objetivo a tipo entero

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear una instancia del clasificador de árbol de decisión
model = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Predecir los valores para el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la métrica F1 y AUC-ROC
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred)

# Mostrar los resultados
print(f"F1 Score: {f1:.2f}")
print(f"AUC-ROC: {auc_roc:.2f}")

# Mostrar un informe de clasificación
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))

F1 Score: 0.48
AUC-ROC: 0.68

Informe de clasificación:
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      1607
           1       0.45      0.51      0.48       393

    accuracy                           0.78      2000
   macro avg       0.67      0.68      0.67      2000
weighted avg       0.79      0.78      0.79      2000



<span style='color:#1E90FF '> Explicación: </span>

Análisis del equilibrio de clases: Detectamos un desequilibrio en los datos, con más clientes que no abandonan que los que sí lo hacen.
División de datos: Separamos los datos en conjuntos de entrenamiento (80%) y prueba (20%) para evaluar el modelo.
Entrenamiento del modelo: Utilizamos un árbol de decisión sin técnicas para manejar el desequilibrio. El modelo se entrenó con los datos de entrenamiento y luego se probaron las predicciones.
Evaluación del rendimiento: Las métricas F1 (0.48) y AUC-ROC (0.68) mostraron que el modelo tiene dificultades para predecir correctamente los casos de abandono, siendo mucho mejor en la clase mayoritaria (clientes que no abandonan).

<span style='color:#FF69B4 '> Mejora del Modelo de Clasificación y realización de prueba final  </span>

<span style='color:#2ca02c '> Pasos: </span>

<span style='color:#ff7f0e '> Paso 1: </span> Corregir el desequilibrio de clases utilizando dos enfoques
Sobremuestreo de la clase minoritaria: Aumentar la cantidad de ejemplos en la clase minoritaria (clientes que abandonan) mediante la duplicación o generación de ejemplos sintéticos.
Ponderación de clases en el modelo: Ajustar los pesos del modelo para que las clases minoritarias tengan más impacto en el entrenamiento.

<span style='color:#ff7f0e '> Paso 2: </span> Entrenar y validar diferentes modelos utilizando los enfoques anteriores
Utilizaremos train_test_split para dividir los datos en conjuntos de entrenamiento y validación.
Entrenaremos modelos con los dos enfoques de balanceo y evaluaremos su rendimiento en el conjunto de validación.

<span style='color:#ff7f0e '> Paso 3: </span> Realizar la prueba final en el conjunto de prueba con el mejor modelo encontrado

<span style='color:#663399 '> Paso 1: </span>

In [7]:
# Combinar X_train e y_train para aplicar el sobremuestreo
train_data = X_train.copy()
train_data['Exited'] = y_train

# Separar la clase mayoritaria y la minoritaria
majority = train_data[train_data['Exited'] == 0]
minority = train_data[train_data['Exited'] == 1]

# Sobremuestrear la clase minoritaria
minority_oversampled = resample(minority, 
                                replace=True,     # Sobremuestrear con reemplazo
                                n_samples=len(majority), # Número de muestras para igualar la clase mayoritaria
                                random_state=42)

# Combinar las clases mayoritaria y minoritaria sobremuestreadas
train_data_balanced = pd.concat([majority, minority_oversampled])

# Separar de nuevo X_train_balanced e y_train_balanced
X_train_balanced = train_data_balanced.drop(columns=['Exited'])
y_train_balanced = train_data_balanced['Exited']

In [8]:
# Configurar el clasificador con la opción 'class_weight' para manejar el desequilibrio
model_weighted = DecisionTreeClassifier(random_state=42, class_weight='balanced')

print(model_weighted)

DecisionTreeClassifier(class_weight='balanced', random_state=42)


<span style='color:#663399 '> Paso 2: </span>

In [9]:
# Entrenar el modelo con los datos sobremuestreados
model_oversampled = DecisionTreeClassifier(random_state=42)
model_oversampled.fit(X_train_balanced, y_train_balanced)

# Predecir en el conjunto de validación
y_pred_oversampled = model_oversampled.predict(X_test)

# Evaluar el rendimiento
f1_oversampled = f1_score(y_test, y_pred_oversampled)
auc_roc_oversampled = roc_auc_score(y_test, y_pred_oversampled)

print(f"Modelo Sobremuestreo - F1 Score: {f1_oversampled:.2f}, AUC-ROC: {auc_roc_oversampled:.2f}")

Modelo Sobremuestreo - F1 Score: 0.45, AUC-ROC: 0.66


In [10]:
# Entrenar el modelo con la ponderación de clases
model_weighted.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_pred_weighted = model_weighted.predict(X_test)

# Evaluar el rendimiento
f1_weighted = f1_score(y_test, y_pred_weighted)
auc_roc_weighted = roc_auc_score(y_test, y_pred_weighted)

print(f"Modelo Ponderación - F1 Score: {f1_weighted:.2f}, AUC-ROC: {auc_roc_weighted:.2f}")

Modelo Ponderación - F1 Score: 0.48, AUC-ROC: 0.68


<span style='color:#1E90FF '> Explicación: </span>

Los resultados muestran que la técnica de ponderación de clases tuvo un mejor desempeño en comparación con el sobremuestreo.
El modelo con ponderación de clases logró un F1 Score y AUC-ROC ligeramente mejores que el modelo con sobremuestreo. Esto sugiere que ajustar los pesos para las clases puede ser más efectivo en este caso para mejorar el rendimiento del modelo en la detección de clientes que abandonan.

<span style='color:#663399 '> Paso 3: </span>

In [11]:
# Asegurarse de que el modelo esté entrenado con los datos de entrenamiento
model_weighted.fit(X_train, y_train)

# Predecir los valores para el conjunto de prueba utilizando el modelo con ponderación
y_pred_final = model_weighted.predict(X_test)

# Calcular las métricas para la prueba final
f1_final = f1_score(y_test, y_pred_final)
auc_roc_final = roc_auc_score(y_test, y_pred_final)

# Mostrar los resultados
print(f"Prueba Final - F1 Score: {f1_final:.2f}")
print(f"Prueba Final - AUC-ROC: {auc_roc_final:.2f}")

# Informe de clasificación final
print("\nInforme de clasificación (Prueba Final):")
print(classification_report(y_test, y_pred_final))

Prueba Final - F1 Score: 0.48
Prueba Final - AUC-ROC: 0.68

Informe de clasificación (Prueba Final):
              precision    recall  f1-score   support

           0       0.87      0.88      0.88      1607
           1       0.49      0.48      0.48       393

    accuracy                           0.80      2000
   macro avg       0.68      0.68      0.68      2000
weighted avg       0.80      0.80      0.80      2000



<span style='color:#1E90FF '> Explicación: </span>

Los resultados de la prueba final muestran que el modelo tiene un buen rendimiento para la clase mayoritaria (clientes que no abandonan) con un F1-score de 0.88, pero sigue teniendo dificultades para predecir correctamente los casos de abandono, con un F1-score de 0.48 para la clase minoritaria.

El AUC-ROC de 0.68 indica una capacidad moderada para distinguir entre las clases. Aunque la técnica de ponderación de clases mejoró el modelo, hay margen para seguir mejorando. Se recomienda explorar algoritmos más complejos o ajustar los hiperparámetros para optimizar la detección de los clientes que podrían abandonar.

# <span style='color:#000080 '> Conclusión </span>

Para abordar el problema de abandono de clientes en Beta Bank, preparamos los datos eliminando columnas irrelevantes y aplicando `One-Hot Encoding` a las variables categóricas. Luego, se investigó el desequilibrio de clases, encontrando que solo el 20% de los clientes había abandonado. Entrenamos un modelo inicial sin corregir el desequilibrio, lo cual confirmó la necesidad de técnicas específicas para mejorar la predicción de la clase minoritaria.

Utilizamos dos enfoques para manejar el desequilibrio: **sobremuestreo** y **ponderación de clases**. Los datos se dividieron adecuadamente para entrenamiento, validación y prueba. El modelo con ponderación de clases mostró un mejor desempeño, con un **F1-score final de 0.48** y un **AUC-ROC de 0.68**, lo que indica una capacidad moderada para distinguir entre las clases.

El proyecto se completó manteniendo una estructura de código clara y documentada, pero aún hay margen para mejorar el rendimiento del modelo mediante algoritmos más avanzados o ajustes adicionales.